## 들어가며
### 선형대수를 왜 알아야 하는가?

DL(Deep Learning)을 이해하기 위해서는 선형대수 + 행렬미분 + 확률의 기초지식이 필요합니다. 최근에 사용되는 Transformer의 경우 attention matrix를 사용하는데 

$$
\operatorname{Att}_{\leftrightarrow}(Q, K, V)=D^{-1} A V, A=\exp \left(Q K^{T} / \sqrt{d}\right), D=\operatorname{diag}\left(A 1_{L}\right)
$$

이러한 경우 행렬에 대한 이해가 필요합니다. 이번 포스팅의 목표는 선형대수와 행렬 미분의 기초를 배우고 간단히 머신러닝 알고리즘(PCA)를 유도 해보고자 합니다.

## 기본 표기법 (Basic Notation)

- $A\in \mathbb{R}^{m\times n}$는 $m$개의 행과 $n$개의 열을 가진 행렬을 의미한다.
- $x \in \mathbb{R}^n$는 $n$개의 원소를 가진 벡터를 의미한다. $n$차원 벡터는 $n$개의 행과 1개의 열을 가진 행렬로 생각할 수도 있다. 이것을 열벡터(column vector)로 부르기도 한다. 만약, 명시적으로 행벡터(row vector)를 표현하고자 한다면, $x^T$($T$는 transpose를 의미)로 쓴다.
- 벡터 $x$의 $i$번째 원소는 $x_i$로 표시한다.


$$
x=\left[\begin{array}{c}
x_{1} \\
x_{2} \\
\vdots \\
x_{n}
\end{array}\right]
$$


- $a_{ij}$(또는 $A_{ij}, A_{i,j}$)는 행렬 $A$의 $i$번째 행, $j$번째 열에 있는 원소를 표시한다.

$$
A=\left[\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1 n} \\
a_{21} & a_{22} & \cdots & a_{2 n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m 1} & a_{m 2} & \cdots & a_{m n}
\end{array}\right]
$$

- $A$의 $j$번째 열을 $a_j$ 혹은 $A_{:,j}$로 표시한다.

$$
A=\left[\begin{array}{cccc}
\mid & \mid & & \mid \\
a_{1} & a_{2} & \cdots & a_{n} \\
\mid & \mid & & \mid
\end{array}\right]
$$

- $A$의 $i$번째 행을 $a_i^T$ 혹은 $A_{i,:}$로 표시한다.

$$
A=\left[\begin{array}{ccc}
- & a_{1}^{T} & - \\
- & a_{2}^{T} & - \\
& \vdots & \\
- & a_{m}^{T} & -
\end{array}\right]
$$


### Python에서 벡터, 행렬의 표현 방법

In [1]:
import numpy as np

In [2]:
x = np.array([10.5, 5.2, 3.25])

In [3]:
x.shape
# (3,)

(3,)

위의 결과 같이 (3,)으로 표시가 된다. 이는 1차원 배열을 나타낸다

`numpy.expand_dims(a, axis)`
- Expand the shape of an array.

`np.expand_dims` 메소드를 사용해서 차원의 크기를 임의로 크게해 수 있습니다. 이는 우리가 이전의 포스팅에서 `x[:, np.newaxis]`으로 차원의 크기를 늘려주었던 방법과 동일한 방법입니다. 또한, `axis` 인자를 통해 열의 방향으로 확장할 것인지, 행의 방향으로 확장할 것인지를 지정 할 수있습니다

In [10]:
np.expand_dims(x, axis=1) # column 방향으로 확장

array([[10.5 ],
       [ 5.2 ],
       [ 3.25]])

In [5]:
np.expand_dims(x, axis=1).shape

(3, 1)

In [7]:
np.expand_dims(x, axis=0) # row 방향으로 확장,  x[np.newaxis, :] or x[np.newaxis]와 동일한 방법

array([[10.5 ,  5.2 ,  3.25]])

In [8]:
np.expand_dims(x, axis=0).shape

(1, 3)

In [11]:
A = np.array([
    [10, 20, 30],
    [40, 50, 60]
])

In [13]:
A.shape

(2, 3)

In [14]:
A[0,2] # equivalent A[0][2]

30

In [16]:
# Column vector
A[:, 1]

array([20, 50])

위 벡터는 column 벡터를 추출하였지만, row 벡터 인 것처럼 보입니다. 이는 단순히 표기를 1차원 배열로 한 것일 뿐입니다.

In [17]:
# Row vector
A[1, :]

array([40, 50, 60])

## 행렬의 곱셈(Matrix Muliplication)

두 개의 행렬 $A\in \mathbb{R}^{m\times n}$, $B\in \mathbb{R}^{n\times p}$의 곱 $C = AB \in \mathbb{R}^{m\times p}$는 다음과 같이 정의된다.

$$C_{ij} = \sum_{k=1}^n A_{ik}B_{kj}$$

행렬의 곱셈을 이해하는 몇 가지 방식들이 존재 합니다. 
- 벡터 $\times$ 벡터
- 행렬 $\times$ 벡터
- 행렬 $\times$ 행렬

### 벡터 $\times$ 벡터 (Vector-Vector Products)

두 개의 벡터 $x, y\in \mathbb{R}^n$이 주어졌을 때 **내적**(inner product 또는 dot product) $x^Ty$는 다음과 같이 정의된다.

$$
x^{T} y \in \mathbb{R}=\left[\begin{array}{llll}
x_{1} & x_{2} & \cdots & x_{n}
\end{array}\right]\left[\begin{array}{c}
y_{1} \\
y_{2} \\
\vdots \\
y_{n}
\end{array}\right]=\sum_{i=1}^{n} x_{i} y_{i}
$$

$$
x^{T} y=y^{T} x
$$

In [18]:
x = np.array([1,2,3])
y = np.array([4,5,6])
x.dot(y)

32

#### 외적
외적을 구할 때는 두 벡터의 차원이 다른 경우입니다. 내적의 경우에는 두 벡터의 차원이 같은 경우입니다.
두 개의 벡터 $x\in \mathbb{R}^m, y\in \mathbb{R}^n$이 주어졌을 때 외적(outer product) $xy^T\in \mathbb{R}^{m\times n}$는 다음과 같이 정의된다.

여기서 기억해야 할 점은 내적을 구했을 때는 스칼라값을 얻을 수 있었지만, 외적을 구했을 떄는 행령을 얻을 수 있습니다.

$$
\left.x y^{T_{*}} \in \mathbb{R}^{m \times n}=\left[\begin{array}{c}
x_{1} \\
x_{2} \\
\vdots \\
x_{m}
\end{array}\right]\left[\begin{array}{lll}
y_{1} & y_{2} & \cdots
\end{array}\right]_{n}\right]=\left[\begin{array}{cccc}
x_{1} y_{1} & x_{1} y_{2} & \cdots & x_{1} y_{n} \\
x_{2} y_{1} & x_{2} y_{2} & \cdots & x_{2} y_{n} \\
\vdots & \vdots & \ddots & \vdots \\
x_{m} y_{1} & x_{m} y_{2} & \cdots & x_{m} y_{n}
\end{array}\right]
$$

In [19]:
x = np.array([1,2,3])
y = np.array([4,5,6])

In [20]:
x = np.expand_dims(x, axis=1) # column 방향으로 추가
y = np.expand_dims(y, axis=0) # row 방향으로 추가
x.shape, y.shape # x: column vector, y: row vector

((3, 1), (1, 3))

In [22]:
x

array([[1],
       [2],
       [3]])

In [23]:
y

array([[4, 5, 6]])

In [24]:
np.matmul(x,y)

array([[ 4,  5,  6],
       [ 8, 10, 12],
       [12, 15, 18]])

#### 외적이 유용한 경우
외적이 유용한 경우는 다음과 같습니다. 행렬 $A$는 모든 colummn이 동일한 벡터 $x$를 가지고 있다고 가정해 봅시다. 외적을 이용하면 간편하게 $x \mathbf{1}^{T}$로 나타낼 수 있습니다. ($1 \in \mathbb{R}^{n}$은 모든 원소가 1인 $n$-차원 벡터)

$$
A=\left[\begin{array}{llll}
\mid & \mid & & \mid \\
x & x & \cdots & x \\
\mid & \mid & & \mid
\end{array}\right]=\left[\begin{array}{cccc}
x_{1} & x_{1} & \cdots & x_{1} \\
x_{2} & x_{2} & \cdots & x_{2} \\
\vdots & \vdots & \ddots & \vdots \\
x_{m} & x_{m} & \cdots & x_{m}
\end{array}\right]=\left[\begin{array}{c}
x_{1} \\
x_{2} \\
\vdots \\
x_{m}
\end{array}\right]\left[\begin{array}{llll}
1 & 1 & \cdots & 1
\end{array}\right]=x \mathbf{1}^{T}
$$

In [27]:
# column 벡터
x = np.expand_dims(np.array([1,2,3]), axis=1)
x

array([[1],
       [2],
       [3]])

In [28]:
ones = np.ones([1,4])

In [31]:
A = np.matmul(x, ones)
A

array([[1., 1., 1., 1.],
       [2., 2., 2., 2.],
       [3., 3., 3., 3.]])

### 행렬 $\times$ 벡터 (Matrix-Vector Products)

행렬 $A\in \mathbb{R}^{m\times n}$와 벡터 $x\in \mathbb{R}^n$의 곱은 벡터 $y = Ax \in \mathbb{R}^m$이다. 이 곱을 몇 가지 측면에서 바라볼 수 있다.

#### 열벡터를 오른쪽에 곱하고($Ax$), $A$가 행의 형태로 표현되었을 때

$$
y=A x=\left[\begin{array}{ccc}
- & a_{1}^{T} & - \\
- & a_{2}^{T} & - \\
& \vdots & \\
- & a_{m}^{T} & -
\end{array}\right] x=\left[\begin{array}{c}
a_{1}^{T} x \\
a_{2}^{T} x \\
\vdots \\
a_{m}^{T} x
\end{array}\right]
$$

In [32]:
A = np.array([
    [1,2,3],
    [4,5,6]
])

In [33]:
ones = np.ones([3,1])
ones

array([[1.],
       [1.],
       [1.]])

In [34]:
np.matmul(A, ones)

array([[ 6.],
       [15.]])

#### column 벡터를 오른쪽에 곱하고, $A$가 Column의 형태로 표현 되었을 때

$$
y=A x=\left[\begin{array}{cccc}
\mid & \mid & & 1 \\
a_{1} & a_{2} & \cdots & a_{n} \\
\mid & \mid & & \mid
\end{array}\right]\left[\begin{array}{c}
x_{1} \\
x_{2} \\
\vdots \\
x_{n}
\end{array}\right]=\left[\begin{array}{c}
1 \\
a_{1} \\
1
\end{array}\right] x_{1}+\left[\begin{array}{c}
1 \\
a_{2} \\
1
\end{array}\right] x_{2}+\cdots+\left[\begin{array}{c}
1 \\
a_{n} \\
1
\end{array}\right] x_{n}
$$

In [36]:
A = np.array([
    [1,0,1],
    [0,1,1]
])

x = np.array([
    [1],
    [2],
    [3]
])
np.matmul(A, x)

array([[4],
       [5]])

In [37]:
for i in range(A.shape[1]):
    print('a_'+str(i)+':', A[:,i], '\tx_'+str(i)+':', x[i], '\ta_'+str(i)+'*x_'+str(i)+':', A[:,i]*x[i])

a_0: [1 0] 	x_0: [1] 	a_0*x_0: [1 0]
a_1: [0 1] 	x_1: [2] 	a_1*x_1: [0 2]
a_2: [1 1] 	x_2: [3] 	a_2*x_2: [3 3]
